In [53]:
import pandas as pd

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [55]:
# Cargar los datos
df = pd.read_parquet('coffee_db.parquet')

In [1]:
#df.head()

In [57]:
# Cargar los datos
df = pd.read_parquet('coffee_db.parquet')

# Columnas a mantener fijas
id_vars = ['Country', 'Coffee type', 'Total_domestic_consumption']

# Columnas de años que se van a transformar en filas
value_vars = df.columns.drop(id_vars)

# Transformar de formato ancho a largo
df_long = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='Year', value_name='Consumption')

# Limpiar y convertir la columna de año
df_long['Year'] = df_long['Year'].str.slice(0, 4).astype(int)

# Renombrar columnas para mejor manejo
df_long.rename(columns={
    'Country': 'Pais',
    'Coffee type': 'Tipo_Cafe',
    'Total_domestic_consumption': 'Consumo_Total_Original',
    'Consumption': 'Consumo'
}, inplace=True)

In [58]:
# Verificar la consistencia de la columna de consumo total
verification = df_long.groupby('Pais')['Consumo'].sum().reset_index()
merged = pd.merge(verification, df[['Country', 'Total_domestic_consumption']].drop_duplicates(), left_on='Pais', right_on='Country')

# La función np.allclose compara si dos arrays son iguales elemento por elemento
are_totals_consistent = np.allclose(merged['Consumo'], merged['Total_domestic_consumption'])

print(f"¿Son consistentes los totales calculados con la columna original?: {are_totals_consistent}")

¿Son consistentes los totales calculados con la columna original?: True


In [59]:
def plot_data(data, x, y, kind='bar', title='', xlabel='', ylabel='', add_labels=True):
    """Crea un gráfico de barras o líneas con diseño mejorado y etiquetas en Millones (M)."""
    fig, ax = plt.subplots()

    if kind == 'bar':
        sns.barplot(x=x, y=y, data=data, ax=ax, palette='viridis', edgecolor='black')
    elif kind == 'line':
        sns.lineplot(x=x, y=y, data=data, ax=ax, marker='o', color='dodgerblue', linewidth=2.5, markersize=7)

    ax.set_title(title, pad=20)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    ax.set_xticks(data[x].unique())
    ax.tick_params(axis='x', rotation=70)
    sns.despine()

    if add_labels:
        if kind == 'bar':
            for p in ax.patches:
                value = p.get_height()
                if 'Percent' in y:
                    label = f'{value:.1f}%'
                else:
                    # Formato en Millones (M) sin decimales
                    label = f'{value/1e6:.0f}M'
                ax.annotate(label, (p.get_x() + p.get_width() / 2., value),
                            ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)
        
        elif kind == 'line' and ('Tipo_Cafe' not in data.columns or data['Tipo_Cafe'].nunique() == 1):
            for index, row in data.iterrows():
                value = row[y]
                # --- MODIFICACIÓN CLAVE: Etiqueta en Millones (M) ---
                label = f'{value/1e6:.0f}M'
                ax.text(row[x], value + (ax.get_ylim()[1] * 0.015), label,
                        ha='center', va='bottom', fontsize=9, color='black')

    plt.tight_layout()
    plt.show()

In [60]:
# Celda de Estilo (ejecutar una vez al principio)
# Establece un tema visual atractivo con fondo cuadriculado y un tamaño de figura por defecto más grande.
sns.set_theme(
    style="whitegrid",
    rc={
        "figure.figsize": (16, 8),
        "axes.titlesize": 18,
        "axes.labelsize": 14,
        "xtick.labelsize": 12
    }
)

In [ ]:
# Agrupar por año y sumar el consumo global
consumo_global_anual = df_long.groupby('Year')['Consumo'].sum().reset_index()

# Graficar la tendencia global
plot_data(consumo_global_anual, 
          x='Year', 
          y='Consumo', 
          kind='line', 
          title='Consumo Mundial de Café (1990-2019)', 
          xlabel='Año', 
          ylabel='Consumo (en tazas)',
          add_labels=True)

In [ ]:
# Calcular el consumo total por país y obtener el top 10
top_10_paises = df_long.groupby('Pais')['Consumo'].sum().nlargest(10).reset_index()

# Graficar el top 10
plot_data(top_10_paises, 
          x='Pais', 
          y='Consumo', 
          title='Top 10 Países por Consumo Total de Café (1990-2019)', 
          xlabel='País', 
          ylabel='Consumo Acumulado (en tazas)')

In [63]:
def calculate_cagr(df, min_years=15):
    """Calcula la tasa de crecimiento anual compuesta para cada país,
       filtrando por un mínimo de años para mayor robustez."""
    cagr_data = df.sort_values('Year').groupby('Pais').agg(
        start_year=('Year', 'min'),
        end_year=('Year', 'max'),
        start_value=('Consumo', 'first'),
        end_value=('Consumo', 'last')
    ).reset_index()

    # Evitar división por cero
    cagr_data = cagr_data[cagr_data['start_value'] > 0]
    
    # Calcular el número de años
    cagr_data['num_years'] = cagr_data['end_year'] - cagr_data['start_year']
    
    # --- MODIFICACIÓN CLAVE: Filtrar por un rango de años mínimo ---
    # Esto evita que un pico de 1 o 2 años dé un CAGR engañosamente alto
    cagr_data = cagr_data[cagr_data['num_years'] >= min_years]

    # Calcular CAGR
    cagr_data['CAGR'] = ((cagr_data['end_value'] / cagr_data['start_value']) ** (1 / cagr_data['num_years'])) - 1
    
    return cagr_data.sort_values('CAGR', ascending=False)

In [64]:
def plot_data(data, x, y, kind='bar', title='', xlabel='', ylabel='', add_labels=True):
    """Crea un gráfico de barras o líneas con diseño mejorado y etiquetado en todos los puntos."""
    fig, ax = plt.subplots()
    
    hue_col = 'Tipo_Cafe' if 'Tipo_Cafe' in data.columns else None

    if kind == 'bar':
        sns.barplot(x=x, y=y, data=data, ax=ax, palette='viridis', edgecolor='black')
    elif kind == 'line':
        palette = 'tab10' if hue_col and data[hue_col].nunique() > 1 else 'dodgerblue'
        sns.lineplot(x=x, y=y, data=data, ax=ax, marker='o', hue=hue_col, palette=palette, linewidth=2.5)

    ax.set_title(title, pad=20)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.tick_params(axis='x', rotation=70)

    if kind == 'line':
        ax.set_xticks(data[x].unique())
        
    sns.despine()

    if add_labels:
        if kind == 'bar':
            for p in ax.patches:
                value = p.get_height()
                if 'Percent' in y: label = f'{value:.1f}%'
                else: label = f'{value/1e6:.0f}M'
                ax.annotate(label, (p.get_x() + p.get_width() / 2., value),
                            ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)
        
        elif kind == 'line':
            # --- CORRECCIÓN: Etiquetar TODOS los puntos, siempre ---
            y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
            offset = y_range * 0.02 # Un pequeño offset vertical

            for index, row in data.iterrows():
                value = row[y]
                label = f'{value/1e6:.0f}M'
                # Usamos un tamaño de fuente más pequeño para que no se sature
                ax.text(row[x], value + offset, label,
                        ha='center', va='bottom', fontsize=8, color='black')

    # Asegurarse de que la leyenda se muestre si hay múltiples líneas
    if hue_col and data[hue_col].nunique() > 1:
        ax.legend(title=hue_col)
    else:
        # Ocultar la leyenda si solo hay una línea
        if ax.get_legend() is not None:
            ax.get_legend().remove()
            
    plt.tight_layout()
    plt.show()

In [ ]:
# Celda 11:
# Calcular CAGR (ahora más robusto) y obtener el top 10
mercados_emergentes = calculate_cagr(df_long, min_years=15)
top_10_crecimiento = mercados_emergentes.nlargest(10, 'CAGR')
top_10_crecimiento['CAGR_Percent'] = top_10_crecimiento['CAGR'] * 100

# Graficar el top 10 de crecimiento
plot_data(top_10_crecimiento, 
          x='Pais', 
          y='CAGR_Percent',
          kind='bar',
          title='Top 10 Mercados por Tasa de Crecimiento (CAGR > 15 años)', 
          xlabel='País', 
          ylabel='Tasa de Crecimiento Anual (%)')

# Celda 13:
# Agrupar por año y tipo de café
consumo_por_tipo = df_long.groupby(['Year', 'Tipo_Cafe'])['Consumo'].sum().reset_index()

# Graficar las tendencias (ahora con etiquetas al final de cada línea)
plot_data(consumo_por_tipo, 
          x='Year', 
          y='Consumo', 
          kind='line',
          title='Consumo Mundial por Tipo de Café', 
          xlabel='Año', 
          ylabel='Consumo (en tazas)',
          add_labels=True)